In [ ]:
import numpy as np
import scipy as sp
import os 
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
import importlib

In [ ]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu
import my_classification_utils as mcu
#importlib.reload(mdu)

In [ ]:
features = mfileu.read_file('features', 'lab_features_combo_right.pkl')
labels = mfileu.read_file('ssilv', 'lab_ssilv_array_combo.pkl')

In [ ]:
def get_lopo_train_features_labels(features, exclude_subj):    
    train_x, train_y = [], []
    for subj in range(len(features)):        
        if subj==exclude_subj:
            continue
        for sess in range(len(features[subj])):             
            f = features[subj][sess][:, 1:]
            #a = data[subj][sess]["annots"]            
            #l = mslabu.get_labels_right_hand(features[subj][sess][:, 0], a, 5*16)
            l = labels[(subj,sess)][:, 1]
            
            assert len(f)==len(l)
            assert np.sum(features[subj][sess][:, 0] - labels[(subj,sess)][:, 0]) ==0
            
            train_x, train_y = (f, l) if len(train_x)==0 else ( np.concatenate((train_x, f)), np.concatenate((train_y, l)) )
    
    return train_x, train_y
    

In [ ]:
#for subj in [0, 1, 2, 3, 4, 5, 6, 100]:
subj = int(sys.argv[1])
print("Train Subject: ", subj)

train_x, train_y = get_lopo_train_features_labels(features=features, exclude_subj=subj)
print("Subj:", subj, ", shapes:", train_x.shape, train_y.shape, "Neg, Bite, sip, other:", np.sum(train_y==0), np.sum(train_y==1), np.sum(train_y==2), np.sum(train_y==3))
train_y[train_y==2] = 0
print("Subj:", subj, ", shapes:", train_x.shape, train_y.shape, "Neg, Bite, sip, other:", np.sum(train_y==0), np.sum(train_y==1), np.sum(train_y==2), np.sum(train_y==3))

#clf = RandomForestClassifier(n_estimators =100, random_state=0, n_jobs=-1, verbose=1)    
clf = SVC(verbose=True)    

clf.fit(train_x, train_y)    
print("Training done!")

mfileu.write_file('bite_models_SVM', 'lab_'+str(subj)+'.pkl', clf)
print("Model saved")
